In [12]:
import numpy as np
import cv2
import skimage.io as io
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)
#Utility
import urllib
import os
import shutil
import time
from PIL import Image

#Vizualization
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import cv2

#MODEL
import tensorflow as tf
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout, AveragePooling2D, Concatenate,Lambda
from tensorflow.keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import preprocess_input


In [13]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [14]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [15]:
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]

(5, array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

In [16]:
X_train=tf.keras.utils.normalize(X_train,axis=1)
X_test=tf.keras.utils.normalize(X_test,axis=1)

In [17]:
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [18]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
test_gen = ImageDataGenerator()
train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)
STEP_SIZE_TRAIN = int(np.ceil(train_generator.n / train_generator.batch_size))
STEP_SIZE_Test = int(np.ceil(test_generator.n / test_generator.batch_size))

print("Train step size:", STEP_SIZE_TRAIN)
print("Test step size:", STEP_SIZE_Test)

train_generator.reset()
test_generator.reset()

Train step size: 938
Test step size: 157


In [22]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu))
model.add(tf.keras.layers.BatchNormalization(axis=-1))
model.add(tf.keras.layers.Conv2D(64,(3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(32,(3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.BatchNormalization(axis=-1))
model.add(tf.keras.layers.Conv2D(64,(3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.Conv2D(64,(3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization(axis=-1))
model.add(tf.keras.layers.Dense(512,activation=tf.nn.relu))
model.add(tf.keras.layers.BatchNormalization(axis=-1))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax))
#cnnmodel.summary()

In [23]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])

In [24]:
model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,
                                         validation_data=test_generator,validation_steps=STEP_SIZE_Test,epochs=5)

Epoch 1/5
938/938 [==============================] - 134s 142ms/step - loss: 0.3392 - accuracy: 0.9002 - val_loss: 0.0539 - val_accuracy: 0.9840
Epoch 2/5
938/938 [==============================] - 173s 184ms/step - loss: 0.0795 - accuracy: 0.9757 - val_loss: 0.0527 - val_accuracy: 0.9836
Epoch 3/5
938/938 [==============================] - 186s 198ms/step - loss: 0.0569 - accuracy: 0.9831 - val_loss: 0.0392 - val_accuracy: 0.9880
Epoch 4/5
938/938 [==============================] - 182s 194ms/step - loss: 0.0525 - accuracy: 0.9839 - val_loss: 0.0746 - val_accuracy: 0.9738
Epoch 5/5
938/938 [==============================] - 180s 192ms/step - loss: 0.0417 - accuracy: 0.9869 - val_loss: 0.0613 - val_accuracy: 0.9817


In [25]:
# Save model
model.save('mnist_classification.h5')

In [15]:
model.save_weights("Model_mnist.h5")

In [ ]:
predictions = model.predict_classes(X_test)